In [ ]:
import openai
import os

openai.__version__

In [ ]:
jsts_url = "https://raw.githubusercontent.com/yahoojapan/JGLUE/main/datasets/jsts-v1.1/valid-v1.1.json"
jsick_url = "https://github.com/verypluming/JSICK/raw/main/jsick/test.tsv"
miracle_n_hard_negs = 300
miracle_n_recall = 30
model_id = ""
dimension = -1
query_prefix = ""
passage_prefix = ""

# Model

In [ ]:
import numpy as np
import dotenv

dotenv.load_dotenv("openai_key", override=True)
client = openai.OpenAI()


# https://platform.openai.com/docs/guides/embeddings/use-cases
def get_embedding(texts):
    texts = [text.replace("\n", " ")[:2000] for text in texts]
    if "text-embedding-3" not in model_id:
        return client.embeddings.create(input=texts, model=model_id).data
    else:
        return client.embeddings.create(
            input=texts, model=model_id, dimensions=dimension
        ).data


def get_embeddings(texts):
    all_embeddings = []
    batch_size = 100
    for i in range(0, len(texts), batch_size):
        all_embeddings += [
            emb.embedding for emb in get_embedding(texts[i : i + batch_size])
        ]
    return np.array(all_embeddings)

# JSTS

In [ ]:
import json
import pandas as pd
from urllib.request import urlopen

df = pd.DataFrame([json.loads(line) for line in urlopen(jsts_url).readlines()])
df.head(1)

In [ ]:
df.shape

## Encode

In [ ]:
import numpy as np


sentence1_embs = get_embeddings(df["sentence1"].values)
sentence2_embs = get_embeddings(df["sentence2"].values)


sentence1_embs.shape, sentence2_embs.shape

## Correlation Score

In [ ]:
from scipy.spatial.distance import cosine, euclidean
from scipy.stats import spearmanr

df["similarity"] = [
    1 - cosine(s1, s2) for s1, s2 in zip(sentence1_embs, sentence2_embs)
]
jsts_score = spearmanr(df["similarity"], df["label"])[0]
jsts_score

# JSICK

In [ ]:
df = pd.read_csv(jsick_url, sep="\t")
df.head(1)

In [ ]:
df.shape

## Encode

In [ ]:
sentence1_embs = get_embeddings(df["sentence_A_Ja"].values)
sentence2_embs = get_embeddings(df["sentence_B_Ja"].values)
sentence1_embs.shape, sentence2_embs.shape

## Correlation Score

In [ ]:
from scipy.spatial.distance import cosine
from scipy.stats import spearmanr

df["similarity"] = [
    1 - cosine(s1, s2) for s1, s2 in zip(sentence1_embs, sentence2_embs)
]
jsick_score = spearmanr(df["similarity"], df["relatedness_score_Ja"])[0]
jsick_score

# Miracle
* Need access token for huggingface

In [ ]:
import os
import dotenv

dotenv.load_dotenv("huggingface_access_token", override=True)

In [ ]:
import datasets

# query and positives
ds = datasets.load_dataset(
    "miracl/miracl", "ja", use_auth_token=os.environ["HF_ACCESS_TOKEN"], split="dev"
)
ds

In [ ]:
# all corpus texts
corpus = datasets.load_dataset("miracl/miracl-corpus", "ja")
corpus

In [ ]:
# hard negatives
with open("./miracl_hard_negs_1000.json") as f:
    hn = json.loads(f.read())
len(hn), list(hn.keys())[:5], hn["0"].keys(), hn["0"]["docids"][:2], hn["0"]["indices"][
    :2
]

In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist


def get_text(corpus_item):
    return corpus_item["title"] + " " + corpus_item["text"]


corpus_dict = {item["docid"]: get_text(item) for item in corpus["train"]}

In [ ]:
n_total_pos = 0
n_total_tp = 0

# only evaluate first 100 queries
for item in ds.select(range(100)):
    # query
    query_emb = get_embeddings([query_prefix + item["query"]])

    # passages are set(300 hard negatives + positives)
    positive_docids = [pp["docid"] for pp in item["positive_passages"]]
    positive_texts = [get_text(pp) for pp in item["positive_passages"]]
    hn_docids = hn[item["query_id"]]["docids"][:miracle_n_hard_negs]

    # drop hard negatives in positives
    hn_docids = [docid for docid in hn_docids if docid not in positive_docids]

    # search target
    target_docids = positive_docids + hn_docids
    target_texts = positive_texts + [corpus_dict[docid] for docid in hn_docids]

    # embedding
    tmppath = f'tmp/{model_id}_{dimension}_{item["query_id"]}.npy'
    if os.path.exists(tmppath):
        target_embs = np.load(tmppath)
    else:
        # use cache
        target_embs = get_embeddings([passage_prefix + text for text in target_texts])
        np.save(tmppath, target_embs)

    # topK
    topk_indices = np.argsort(cdist(query_emb, target_embs, metric="cosine"))[0][
        :miracle_n_recall
    ]

    n_pos = len(positive_docids)
    n_tp = len(
        set(topk_indices) & set(range(len(positive_docids)))
    )  # positives are first indices

    n_total_pos += n_pos
    n_total_tp += n_tp

miracl_recall = n_total_tp / n_total_pos

n_total_pos, n_total_tp, miracl_recall

# Output

In [ ]:
model_id, dimension, jsts_score, jsick_score, miracl_recall

In [ ]:
import json

with open(f'./scores/{model_id.replace("/", "_")}_{dimension}.txt', "w") as f:
    f.write(
        json.dumps(
            {
                "model_id": model_id,
                "jsts": jsts_score,
                "jsick": jsick_score,
                "miracl": miracl_recall,
            }
        )
    )